In [42]:
!pip install Pillow
!pip install tensorflow

In [43]:
mapping = {0: "Nucleoplasm",
1: "Nuclear membrane",
2: "Nucleoli",
3: "Nucleoli fibrillar center",
4: "Nuclear speckles",
5: "Nuclear bodies",
6: "Endoplasmic reticulum",
7: "Golgi apparatus",
8: "Intermediate filaments",
9: "Actin filaments",
10: "Microtubules",
11: "Mitotic spindle",
12: "Centrosome",
13: "Plasma membrane",
14: "Mitochondria",
15: "Aggresome",
16: "Cytosol",
17: "Vesicles and punctate cytosolic patterns",
18: "Negative"}

In [44]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [45]:
df = pd.read_csv("../input/hpa-single-cell-image-classification/train.csv")
fin_df = df[~df['Label'].str.contains('\|')]
fin_df.head(5)

,ID,Label
5,5e22a522-bb99-11e8-b2b9-ac1f6b6435d0,0
6,5f79a114-bb99-11e8-b2b9-ac1f6b6435d0,14
9,5c801c04-bb99-11e8-b2b9-ac1f6b6435d0,14
10,5e9afd56-bb99-11e8-b2b9-ac1f6b6435d0,0
11,5f1af6b4-bb99-11e8-b2b9-ac1f6b6435d0,3


In [46]:
image_path = "../input/hpa-single-cell-image-classification/train"
dic_id_labels = dict(zip(fin_df['ID'], fin_df['Label']))
image_files = os.listdir("../input/hpa-single-cell-image-classification/train")
image_ID_list = [e.split('_')[0] for e in image_files if e.split('_')[0] in dic_id_labels]
image_name_list = [e for e in image_files if e.split('_')[0] in dic_id_labels]
label_list = [dic_id_labels[key] for key in image_ID_list]

In [47]:
def load_images(image_list, labels,batch_size):
    size = len(image_list) // batch_size
    image_name_list = [cv2.resize(cv2.imread(os.path.join(image_path,e)), (600,600),interpolation = cv2.INTER_AREA)
                       for e in image_list[:size]]
    image_size_list = np.array([img.reshape(img.shape[0]*img.shape[1]*img.shape[2]) for img in image_name_list])
    label_list = np.array(labels[:size])
    return image_size_list, label_list.reshape(label_list.shape[0],)

image_subset_list, label_subset_list = load_images(image_name_list, label_list, 400)

In [48]:
# image_subset_list[0].shape, label_subset_list.shape
image_subset_list.shape

(104, 1080000)

Appying supervised learning algorithm logistic regression and random forest on 80 saamples of training set and 20 samples of test set

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(image_subset_list, label_subset_list, test_size=0.2, random_state=42)

In [50]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

LogisticRegression()

In [51]:
predictions = logmodel.predict(X_test)
predictions

array(['7', '16', '14', '16', '3', '7', '14', '14', '16', '16', '14', '7',
       '4', '7', '4', '12', '14', '16', '7', '7', '14'], dtype='<U2')

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.17      0.33      0.22         3
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.05        21
   macro avg       0.02      0.03      0.02        21
weighted avg       0.02      0.05      0.03        21



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
score = accuracy_score(y_test, predictions)
score

0.047619047619047616

The accuracy is low because the data size taken here is very small

In [64]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
clf.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py:320: FutureWarning: The min_impurity_split parameter is deprecated. Its default value has changed from 1e-7 to 0 in version 0.23, and it will be removed in 1.0 (renaming of 0.25). Use the min_impurity_decrease parameter instead.
  FutureWarning
/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py:320: FutureWarning: The min_impurity_split parameter is deprecated. Its default value has changed from 1e-7 to 0 in version 0.23, and it will be removed in 1.0 (renaming of 0.25). Use the min_impurity_decrease parameter instead.
  FutureWarning
/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py:320: FutureWarning: The min_impurity_split parameter is deprecated. Its default value has changed from 1e-7 to 0 in version 0.23, and it will be removed in 1.0 (renaming of 0.25). Use the min_impurity_decrease parameter instead.
  FutureWarning
/opt/conda/lib/python3.7/site-packages/sklearn/tree/_classes.py:320: Futu

RandomForestClassifier(min_impurity_split=1e-07, n_estimators=10, n_jobs=1)

In [65]:
from sklearn.metrics import accuracy_score
preds = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test,preds))

Accuracy: 0.23809523809523808


The accuracy is low because the data size taken here is very small

Appying Deep learning using keras

In [66]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [67]:
num_classes = 10
input_shape = (32, 32, 3)

# Load the train and test data splits
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Display shapes of train and test datasets
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

170500096/170498071 [==============================] - 4s 0us/step
x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [68]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.02),
        layers.experimental.preprocessing.RandomWidth(0.2),
        layers.experimental.preprocessing.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

In [69]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

Model: "cifar10-encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
sequential (Sequential)      (None, None, None, 3)     7         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2048)              23564800  
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [70]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [71]:
learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 5
dropout_rate = 0.5
temperature = 0.05

In [72]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    return model

In [73]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
cifar10-encoder (Functional) (None, 2048)              23564807  
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
Total params: 23,827,079
Trainable params: 23,781,632
Non-trainable params: 45,447
_________________________________________________________________
Epoch 1/5
189/189 [==============================] - 2846s 15s/step - loss: 5.6056
Epoch 2/5
189/189 [==============================] - 2860s 15s/step - loss: 5.1906
Epoch 3/5
189/189 [==============================] - 2755s 15s/step - loss: 5.0599
Epoch 4/5
189/189 [==============================] - 2782s 15s/step - loss: 4.9513
Epoch 5/5
189

In [74]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

Epoch 1/5
189/189 [==============================] - 114s 586ms/step - loss: 2.4250 - sparse_categorical_accuracy: 0.5343
Epoch 2/5
189/189 [==============================] - 107s 564ms/step - loss: 1.2321 - sparse_categorical_accuracy: 0.5822
Epoch 3/5
189/189 [==============================] - 110s 584ms/step - loss: 1.2071 - sparse_categorical_accuracy: 0.5915
Epoch 4/5
189/189 [==============================] - 110s 580ms/step - loss: 1.1951 - sparse_categorical_accuracy: 0.5916
Epoch 5/5
313/313 [==============================] - 24s 73ms/step - loss: 1.1908 - sparse_categorical_accuracy: 0.5931
Test accuracy: 59.31%
